****Using CleF - Climate Finder to discover ESGF data at NCI****

This notebook shows examples of how to use the CleF (Climate Finder) python module to search for ESGF data on the NCI server.
Currently the tool is set up for CMIP5 and CMIP6 data, but other ESGF dataset like CORDEX will be available in the future. 

CleF is installed in the conda module analysis3. This is managed by the CMS and is available simply by running
    module use /g/data3/hh5/public/modules
    module load conda/analysis3  
    
You could use the module interactively, for the moment we will use its command line options.
Let's start!

***Checking the usage***

In [15]:
! ~/.local/bin/clef

Usage: clef [OPTIONS] COMMAND [ARGS]...

Options:
  --remote   returns only ESGF search results
  --local    returns only local files matching ESGF search
  --missing  returns only missing files matching ESGF search
  --request  send NCI request to download missing files matching ESGF search
  --help     Show this message and exit.

Commands:
  cmip5  Search local database for files matching the...
  cmip6  Search local database for files matching the...


By simpling running the command *clef* with no arguments, the tool shows the help message and then exits, basically it is equivalent to *clef --help* . 
We can see currently there are sub-commands, one for each available dataset: *cmip5* and *cmip6*.  
There are also five different options that can be passed before the sub-commands, one we have already seen is *help*. The others are used to modify how the output of the search. 
Lets start from searching some CMIP5 data, to see what we can pass to the *cmip5* sub-command we can simply run it with its *--help* option.

***CMIP5***

In [4]:
! ~/.local/bin/clef cmip5 --help

Usage: clef cmip5 [OPTIONS] [QUERY]...

  Search local database for files matching the given constraints

  Constraints can be specified multiple times, in which case they are
  combined    using OR: -v tas -v tasmin will return anything matching
  variable = 'tas' or variable = 'tasmin'. The --latest flag will check ESGF
  for the latest version available, this is the default behaviour

Options:
  -e, --experiment TEXT           CMIP5 experiment: piControl, rcp85, amip ...
  --experiment_family TEXT        CMIP5 experiment family: Decadal, RCP ...
  -m, --model x                   CMIP5 model acronym: ACCESS1.3, MIROC5 ...
  -t, --table, --mip [Amon|Omon|OImon|LImon|Lmon|6hrPlev|6hrLev|3hr|Oclim|Oyr|aero|cfOff|cfSites|cfMon|cfDay|cf3hr|day|fx|grids]
  -v, --variable x                Variable name as shown in filanames: tas,
                                  pr, sic ...
  -en, --ensemble, --member TEXT  CMIP5 ensemble member: r#i#p#
  --frequency [mon|day|3hr|6hr|fx|yr|monClim|subhr]
 

**Passing arguments and options**

The *help* shows all the constraints we can pass to the tool, there are also some additional options which can change the way we run our search. For the moment we can ignore these and use their default values. Some of the constraints can be passed using an abbreviation,like *-v* instead of *--variable*. This is handy once you are more familiar with the tool. The same option can have more than one name, for example *--ensemble* can also be passed as *--member*, this is because the terminology has changed between cmip5 and cmip6. You can pass how many constraints you want and pass the same constraint more than once. Let s see what happens though if we do not pass any constraint.

In [18]:
! ~/.local/bin/clef cmip5

ERROR: Too many results (3733770), try limiting your search https://esgf-node.llnl.gov/search/cmip5?distrib=True&replica=False&project=CMIP5


In [7]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historicals --table day --ensemble r2i1p1

ERROR: No matches found on ESGF, check at https://esgf-node.llnl.gov/search/cmip5?query=&distrib=True&replica=False&project=CMIP5&ensemble=r2i1p1&experiment=historicals&cmor_table=day&variable=tasmin


Oops that wasn't reasonable! I mispelled the experiment "historicals" does not exists and the tool is telling me it cannot find any matches.

In [8]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historical --table days --ensemble r2i1p1

Usage: clef cmip5 [OPTIONS] [QUERY]...

Error: Invalid value for "--table" / "--mip" / "-t": invalid choice: days. (choose from Amon, Omon, OImon, LImon, Lmon, 6hrPlev, 6hrLev, 3hr, Oclim, Oyr, aero, cfOff, cfSites, cfMon, cfDay, cf3hr, day, fx, grids)


Made another spelling mistake, in this case the tool knows that I passed a wrong value and lists for me all the available options for the CMOR table. Eventually we are aiming to validate all the arguments we can, although for some it is no possible to pass all the possible values (ensemble for example).

In [9]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historical --table day --ensemble r2i1p1

/g/data1b/al33/replicas/CMIP5/combined/CNRM-CERFACS/CNRM-CM5/historical/day/atmos/day/r2i1p1/v20120703/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/IPSL/IPSL-CM5A-LR/historical/day/atmos/day/r2i1p1/v20130506/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/IPSL/IPSL-CM5A-MR/historical/day/atmos/day/r2i1p1/v20130506/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/MIROC/MIROC4h/historical/day/atmos/day/r2i1p1/v20120628/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/MIROC/MIROC5/historical/day/atmos/day/r2i1p1/v20120710/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/MOHC/HadCM3/historical/day/atmos/day/r2i1p1/v20140110/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/MOHC/HadGEM2-CC/historical/day/atmos/day/r2i1p1/v20111129/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/MOHC/HadGEM2-ES/historical/day/atmos/day/r2i1p1/v20110418/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r2i1p1/v20111006/tasmin/
/g/data1b/al33/replicas/CMIP5/combined/MPI-M/MPI-ESM-

The tool first search on the ESGF for all the files that match the constraints we passed. It then looks for these file locally and if it finds them it returns their path on raijin.
For all the files it can't find locally, the tool check an NCI table listing the downloads they are working on. Finally it lists missing datasets which are in the download queue, followed by the datasets that are not available locally and no one has yet requested.

The tool list the datasets paths and datset_ids, if you want you can get a more detailed list by file by passing *--format file* .

The search by default returns the latest available version. What if we want to have a look at all the available versions?

In [5]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historical --table Amon -m ACCESS1.0 --all-versions

/g/data1/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-0/historical/mon/atmos/Amon/r1i1p1/v20120115/tasmin/
/g/data1/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-0/historical/mon/atmos/Amon/r1i1p1/v20120727/tasmin/
/g/data1/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-0/historical/mon/atmos/Amon/r2i1p1/v20130726/tasmin/
/g/data1/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-0/historical/mon/atmos/Amon/r3i1p1/v20140402/tasmin/

Everything available on ESGF is also available locally


The option --all-versions is the reverse of --latest, which is also the default, so we get a list of all available versions. <br>
Since all the ACCESS1.0 data is available on NCI (which is the authoritative source for the ACCESS models) the tool doesn't find any missing datasets and let us know about it.

***CMIP6***

In [2]:
! ~/.local/bin/clef cmip6 --help

Usage: clef cmip6 [OPTIONS] [QUERY]...

  Search local database for files matching the given constraints

  Constraints can be specified multiple times, in which case they are
  combined    using OR: -v tas -v tasmin will return anything matching
  variable = 'tas' or variable = 'tasmin'. The --latest flag will check ESGF
  for the latest version available, this is the default behaviour

Options:
  -mip, --activity [AerChemMIP|C4MIP|CDRMIP|CFMIP|CMIP|CORDEX|DAMIP|DCPP|DynVarMIP|FAFMIP|GMMIP|GeoMIP|HighResMIP|ISMIP6|LS3MIP|LUMIP|OMIP|PAMIP|PMIP|RFMIP|SIMIP|ScenarioMIP|VIACSAB|VolMIP]
  -e, --experiment TEXT           CMIP6 experiment, list of available depends
                                  on activity
  --source_type [AER|AGCM|AOGCM|BGC|CHEM|ISM|LAND|OGCM|RAD|SLAB]
  -t, --table x                   CMIP6 CMOR table: Amon, SIday, Oday ...
  -m, --model, --source_id x      CMIP6 model id: GFDL-AM4, CNRM-CM6-1 ...
  -v, --variable x                CMIP6 variable name as in filenames
  

The **cmip6** sub-command works in the same way but some constraints are different. As well as changes in terminology CMIP6 has more attributes (*facets*) that can be used to search. <br>
Examples of these are the *activity* which groups experiments, *resolution* which is an approximation of the actual resolution and *grid*.

***Controlling the ouput: clef options***

In [11]:
! ~/.local/bin/clef --local cmip6 -e 1pctCO2 -t Amon -v tasmax -v tasmin -g gr

/g/data1b/oi10/replicas/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/1pctCO2/r1i1p1f2/Amon/tasmax/gr/v20180626/
/g/data1b/oi10/replicas/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/1pctCO2/r1i1p1f2/Amon/tasmin/gr/v20180626/
/g/data1b/oi10/replicas/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/1pctCO2/r1i1p1f1/Amon/tasmax/gr/v20180727/
/g/data1b/oi10/replicas/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/1pctCO2/r1i1p1f1/Amon/tasmin/gr/v20180727/


In this example we used the *--local* option for the main command **clef** to get only the local matching data path as output. <br> 
Note also that:
- we are using abbreviations for the options where available; 
- we are passing the variable *-v* option twice; 
- we used the CMIP6 specific option *-g/--grid* to search for all data that is not on the model native grid. This doesn't indicate a grid common to all the CMIP6 output only to the model itself, the same is true for member_id and other attributes.

In [13]:
! ~/.local/bin/clef --missing cmip6 -e 1pctCO2 -v clw -v clwvi -t Amon -g gr


Available on ESGF but not locally:
CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.r1i1p1f2.Amon.clw.gr.v20180626
CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.r1i1p1f2.Amon.clwvi.gr.v20180626
CMIP6.CMIP.IPSL.IPSL-CM6A-LR.1pctCO2.r1i1p1f1.Amon.clw.gr.v20180727
CMIP6.CMIP.IPSL.IPSL-CM6A-LR.1pctCO2.r1i1p1f1.Amon.clwvi.gr.v20180727


This time we used the *--missing* option and the tool returned only the results matching the constraints that are available on the ESGF but not locally (we changed variables to make sure to get some missing data back).

In [14]:
! ~/.local/bin/clef --remote cmip6 -e 1pctCO2 -v tasmin -v tasmax -t Amon -g gr

CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.r1i1p1f2.Amon.tasmax.gr.v20180626
CMIP6.CMIP.IPSL.IPSL-CM6A-LR.1pctCO2.r1i1p1f1.Amon.tasmax.gr.v20180727
CMIP6.CMIP.IPSL.IPSL-CM6A-LR.1pctCO2.r1i1p1f1.Amon.tasmin.gr.v20180727
CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.r1i1p1f2.Amon.tasmin.gr.v20180626


The *--remote* option returns the Dataset_ids of the data matching the constraints, regardless that they are available locally or not.

In [16]:
! ~/.local/bin/clef --remote cmip6 -e 1pctCO2 -v tasmin -v tasmax -t Amon -g gr --format file

CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.r1i1p1f2.Amon.tasmax.gr.v20180626.tasmax_Amon_CNRM-CM6-1_1pctCO2_r1i1p1f2_gr_185001-199912.nc
CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.r1i1p1f2.Amon.tasmin.gr.v20180626.tasmin_Amon_CNRM-CM6-1_1pctCO2_r1i1p1f2_gr_185001-199912.nc
CMIP6.CMIP.IPSL.IPSL-CM6A-LR.1pctCO2.r1i1p1f1.Amon.tasmax.gr.v20180727.tasmax_Amon_IPSL-CM6A-LR_1pctCO2_r1i1p1f1_gr_185001-199912.nc
CMIP6.CMIP.IPSL.IPSL-CM6A-LR.1pctCO2.r1i1p1f1.Amon.tasmin.gr.v20180727.tasmin_Amon_IPSL-CM6A-LR_1pctCO2_r1i1p1f1_gr_185001-199912.nc


Running the same command with the option *--format file* after the sub-command, will return the File_ids instead of the default Dataset_ids. Please note that *--local*, *--remote* and *--missing* together with *--request*, which we will look at next, are all options of the main command **clef** and they need to come before any sub-commands.

***Requesting new data***